In [ ]:
is_colab = 'google.colab' in str(get_ipython())
if is_colab:
    !pip install neuron
    !git clone https://github.com/davidfague/Neural-Modeling.git -b cleanup
    %cd Neural-Modeling/scripts
    !pip install neuron_reduce
    !pip install ipympl

In [ ]:
import sys
sys.path.append("../")
sys.path.append("../Modules/")

from Modules.simulation import Simulation
from Modules.cell_builder import SkeletonCell, CellBuilder
from Modules.constants import HayParameters

import os
from neuron import h

from logger import Logger # type: ignore

import numpy as np

In [ ]:
%cd ../scripts

In [ ]:
try:
    os.system("rm -r x86_64/")
except:
    pass

In [ ]:
# LFP

# Define the template for common attributes
common_attributes = {
    'h_tstop': 1000,
    'record_ecp': True
}

# Define the varying attributes for each sim_name, without including 'sim_name' key
varying_attributes = [
    {'base_sim_name': 'Complex', 'reduce_cell_selective': False},
    {'base_sim_name': 'Basals', 'reduce_basals': True},
    {'base_sim_name': 'Tufts', 'reduce_tufts': True},
    {'base_sim_name': 'Obliques', 'reduce_obliques': True},
    {'base_sim_name': 'All', 'reduce_obliques': True, 'reduce_tufts': True, 'reduce_basals': True},
    {'base_sim_name': 'All - entire apic', 'reduce_apic': True, 'reduce_basals': True}
]

# Generate the list of HayParameters with updated sim_names
all_parameters = [
    HayParameters(
        **common_attributes,
        sim_name=attributes['base_sim_name'],
        **{k: v for k, v in attributes.items() if k != 'base_sim_name'}
    )
    for attributes in varying_attributes
]

# Initialize simulation
sim = Simulation(SkeletonCell.Hay, title='LFP')

# Submit jobs to simulation
for parameters in all_parameters:
    sim.submit_job(parameters)

# Remove directory if it exists
try:
    os.system("rm -r x86_64/")
except Exception as e:
    print(f"Error while removing directory: {e}")

# Run the simulation
sim.run()

# Create directory for saving results
output_dir = f"../notebooks/{sim.title}"
os.makedirs(output_dir, exist_ok=True)

# Plot results for each parameter set and save to individual directories
for parameters in all_parameters:
    sim_name_dir = f"{output_dir}/{parameters.sim_name}"
    os.makedirs(sim_name_dir, exist_ok=True)
    plot_command = f"python3 plot_lfp.py -d {parameters.path} -s {sim_name_dir}"
    os.system(plot_command)

# Placeholder for future enhancements
# Example: Compute correlations between LFP heatmaps
# def compute_correlations(parameter_paths):
#     # Logic to compute and plot correlations
#     pass

# all_parameter_paths = [parameters.path for parameters in all_parameters]
# compute_correlations(all_parameter_paths)


: 